In [20]:
# docker run -p 9000:9000 -p 9001:9001 \
#   -e "MINIO_ROOT_USER=minioadmin" \
#   -e "MINIO_ROOT_PASSWORD=minioadmin" \
#   -v minio_data:/data \
#   minio/minio server /data --console-address ":9001"

import os
import pandas as pd
from minio import Minio
from minio.error import S3Error

MINIO_ENDPOINT = "localhost:9000"
MINIO_ACCESS_KEY = "minioadmin"
MINIO_SECRET_KEY = "minioadmin"
MINIO_SECURE = False

try:
    minio_client = Minio(
        endpoint=MINIO_ENDPOINT,
        access_key=MINIO_ACCESS_KEY,
        secret_key=MINIO_SECRET_KEY,
        secure=MINIO_SECURE
    )
    print("✅ Клиент MinIO успешно создан")
    print(f"   Endpoint: {MINIO_ENDPOINT}")
    print(f"   Secure mode: {MINIO_SECURE}")
except Exception as e:
    print(f"❌ Ошибка при создании клиента MinIO: {e}")

✅ Клиент MinIO успешно создан
   Endpoint: localhost:9000
   Secure mode: False


In [21]:
try:
    if minio_client.bucket_exists("test-connection"):
        pass

    buckets = minio_client.list_buckets()
    print("✅ Соединение с MinIO установлено успешно!")
    print("\n📦 Существующие бакеты:")

    if buckets:
        for bucket in buckets:
            print(f"   - {bucket.name} (создан: {bucket.creation_date})")
    else:
        print("   (нет бакетов)")

except S3Error as e:
    print(f"❌ Ошибка MinIO: {e}")
except Exception as e:
    print(f"❌ Ошибка при проверке соединения: {e}")


✅ Соединение с MinIO установлено успешно!

📦 Существующие бакеты:
   - hw-nikita-voronov (создан: 2025-12-15 21:45:50+00:00)


In [22]:
data = {
    'student_id': [101, 102, 103, 104, 105],
    'name': ['Иванов Иван', 'Петров Петр', 'Сидорова Анна', 'Кузнецов Дмитрий', 'Смирнова Ольга'],
    'course': ['Математика', 'Физика', 'Информатика', 'Математика', 'Физика'],
    'grade': [4.5, 3.8, 4.9, 4.2, 3.5],
    'enrollment_date': pd.to_datetime(['2023-09-01', '2023-09-01', '2023-09-15', '2023-09-10', '2023-09-05'])
}

df = pd.DataFrame(data)
print("✅ DataFrame успешно создан:")
print(f"   Размер: {df.shape[0]} строк, {df.shape[1]} столбцов")
print("\n📊 Содержимое DataFrame:")
print(df)
print("\n📈 Статистика по оценкам:")
print(df['grade'].describe())

✅ DataFrame успешно создан:
   Размер: 5 строк, 5 столбцов

📊 Содержимое DataFrame:
   student_id              name       course  grade enrollment_date
0         101       Иванов Иван   Математика    4.5      2023-09-01
1         102       Петров Петр       Физика    3.8      2023-09-01
2         103     Сидорова Анна  Информатика    4.9      2023-09-15
3         104  Кузнецов Дмитрий   Математика    4.2      2023-09-10
4         105    Смирнова Ольга       Физика    3.5      2023-09-05

📈 Статистика по оценкам:
count    5.000000
mean     4.180000
std      0.554076
min      3.500000
25%      3.800000
50%      4.200000
75%      4.500000
max      4.900000
Name: grade, dtype: float64


In [23]:
from io import BytesIO

csv_filename = "students_data.csv"

csv_data = df.to_csv(index=False).encode('utf-8')
csv_stream = BytesIO(csv_data)

try:

    result = minio_client.put_object(
        bucket_name=BUCKET_NAME,
        object_name=csv_filename,
        data=csv_stream,
        length=len(csv_data),
        content_type='text/csv'
    )

    print(f"✅ Файл '{csv_filename}' успешно загружен в бакет '{BUCKET_NAME}'!")
    print(f"   ETag: {result.etag}")
    print(f"   Версия: {result.version_id if hasattr(result, 'version_id') else 'нет'}")

    objects = minio_client.list_objects(BUCKET_NAME)
    print(f"\n📁 Содержимое бакета '{BUCKET_NAME}':")
    for obj in objects:
        print(f"   - {obj.object_name} ({obj.size} bytes)")

except S3Error as e:
    print(f"❌ Ошибка при загрузке файла в MinIO: {e}")
except Exception as e:
    print(f"❌ Неизвестная ошибка: {e}")


✅ Файл 'students_data.csv' успешно загружен в бакет 'hw-nikita-voronov'!
   ETag: 04c41f4645c056ea7f9034c52909be1c
   Версия: None

📁 Содержимое бакета 'hw-nikita-voronov':
   - students_data.csv (361 bytes)
   - students_data.parquet (1796 bytes)


In [24]:
try:
    import io
    parquet_buffer = io.BytesIO()
    df.to_parquet(parquet_buffer, index=False)
    parquet_buffer.seek(0)

    parquet_filename = "students_data.parquet"

    result = minio_client.put_object(
        bucket_name=BUCKET_NAME,
        object_name=parquet_filename,
        data=parquet_buffer,
        length=parquet_buffer.getbuffer().nbytes,
        content_type='application/octet-stream'
    )

    print(f"✅ Файл '{parquet_filename}' успешно загружен в бакет '{BUCKET_NAME}'!")
    print(f"   Размер: {parquet_buffer.getbuffer().nbytes} байт")

except Exception as e:
    print(f"❌ Ошибка при работе с Parquet: {e}")


✅ Файл 'students_data.parquet' успешно загружен в бакет 'hw-nikita-voronov'!
   Размер: 1796 байт


In [25]:
from io import StringIO

try:
    response = minio_client.get_object(BUCKET_NAME, csv_filename)
    csv_data_from_minio = response.read().decode('utf-8')
    df_from_minio = pd.read_csv(StringIO(csv_data_from_minio))

    print(f"✅ Файл '{csv_filename}' успешно загружен из MinIO!")
    print(f"   Размер данных: {len(csv_data_from_minio)} символов")
    print(f"   Загружено строк: {len(df_from_minio)}")

    print("\n📊 Загруженные данные из MinIO:")
    print(df_from_minio)

    response.close()
    response.release_conn()

except S3Error as e:
    print(f"❌ Ошибка MinIO при загрузке файла: {e}")
except Exception as e:
    print(f"❌ Ошибка при обработке загруженных данных: {e}")


✅ Файл 'students_data.csv' успешно загружен из MinIO!
   Размер данных: 258 символов
   Загружено строк: 5

📊 Загруженные данные из MinIO:
   student_id              name       course  grade enrollment_date
0         101       Иванов Иван   Математика    4.5      2023-09-01
1         102       Петров Петр       Физика    3.8      2023-09-01
2         103     Сидорова Анна  Информатика    4.9      2023-09-15
3         104  Кузнецов Дмитрий   Математика    4.2      2023-09-10
4         105    Смирнова Ольга       Физика    3.5      2023-09-05


In [26]:
try:
    stat = minio_client.stat_object(BUCKET_NAME, csv_filename)

    print("📋 Метаданные файла в MinIO:")
    print(f"   Имя файла: {stat.object_name}")
    print(f"   Размер: {stat.size} байт")
    print(f"   Дата изменения: {stat.last_modified}")
    print(f"   Тип контента: {stat.content_type}")
    print(f"   ETag: {stat.etag}")

except S3Error as e:
    print(f"❌ Ошибка при получении информации о файле: {e}")

📋 Метаданные файла в MinIO:
   Имя файла: students_data.csv
   Размер: 361 байт
   Дата изменения: 2025-12-15 21:57:02+00:00
   Тип контента: text/csv
   ETag: 04c41f4645c056ea7f9034c52909be1c
